# Introduction to Prompt Engineering
In this lesson, you will practice essential prompting principles and related tactics to be able to write effective prompts and leverage OpenAI's APIs using their LLMs (large language models).

## Setting up your environment

First, you must install and update the Python libraries required for working with OpenAI APIs

In [ ]:
# suppress the verbose output by usnig the %%capture annotation
%%capture

# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade python-dotenv

Now, load the API key and relevant Python libaries.

In [ ]:
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

To load environment variables, you can use anything you like but I used python-dotenv. Just create a .env file with your OPENAI_API_KEY then load it.

## Making your first API call

Now that we are all set up, let's make a simple API call to OpenAI's  `gpt-4`  model to see how to do it in the simplest way. We will explore more complex techniques and use cases in the lab exercises that follow.

In [ ]:
MODEL = "gpt-4"

response = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an AI research assistant helping scholars understand different topics in a simple manner. You use technical and scientific terms but explain them in a straightforward way, breaking down complex topics into simple points."},
        {"role": "user", "content": "Hello, who are you?"},
        {"role": "assistant", "content": "Hi! I am an AI research assistant. How can I help you?"},
        {"role": "user", "content": "Can you describe the lifecycle of a star like the Sun?"}
    ],
    temperature=0,
)


If the API ran succesfully, you'll NOT see any error messages. Remember, it is not a chatbot agent, but a backend API. You will have to write code to print the putput to a location of your choice (web, app, console, etc.) and it will not be done automatically.

So, let's print the response:

In [ ]:
response.choices[0].message.content

If you think that the response is not properly formatted and your users will ahve a hard time understanding it, you're right.

So, why not we beautify the response by formatting it properly?

In [ ]:
IPython.display.Markdown(response.choices[0].message.content)

## A Helper Function

Everytime we wish to write prompt, invoke the model API, receive the output, and format the output, we will have repeat the same process again. As this will a typicall API call, let's make a helper function to make live simpler - we'll call this helpr function `get_response()`

This helper function will allow us to easily use prompts and print the generated outputs.

In [ ]:
def get_response(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Controls the randomness of the model's output
    )
    return response.choices[0].message["content"]


## Prompting Best-practices to Follow

Like any conversation, prompti engineering also follows only two essential best practices. These are:

1.   Write clear and specific instructions
2.   Give the model time to “think”

All other principles and best practices emerge from these two only. Let's see them one by one to understand how they can help us become better prompt engineering developers.

### 1: Writing clear and specific instructions

Clear and specific instructions can be written if you follow a few tips. We list the most important ones for you to practice.

#### Tip #1: Use delimiters
Delimiters like **`** , **"**,  **""",** **< >**, **\<tag> \</tag>**, **:**, etc. -- to clearly indicate distinct parts of the input. A delimiter can be anything that separates one part of your input form other. For example, your prompt may look like

`text = f"""Some long text you wish to summarize"""`

`prompt = f"""Summarize the text delimited by triple backticks into a single sentence. ```{text}```""" `

Let's see this with the help of an example.

In [ ]:
text = f"""
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

response = get_response(prompt)

# Beuatify the response
IPython.display.Markdown(response)



#### Tip #2: Request for a structured output
Your OpenAI model is capable of prodcing structured textual output that can easily be parsed by other software. it can produce the response in structured formats such as JSON, XML, HTML, or the plain-vanilla CSV.

If you request your model to return the response in a structured format, you'll be doing your team a great service! Let's check this with an example.


In [ ]:
prompt = f"""
Generate a list of three best selling books for children \
in the age group of twelve to fifteen \
with their authors, country, year of publishing, and a brief summary within 50 words.
Provide them in JSON format with the following keys:
ISBN, title, author, genre, summary.
"""

response = get_response(prompt)

IPython.display.Markdown(response)

#### Tip #3: Ask the model to check whether conditions are satisfied

Like any intelligent helper, you can direct your model to see if the reposne satisfies the conditions you've set out in the prompt. And like all intelligent helpers, the model will make a best effort to comply.

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

STEPS
#1 - ...
#2 - ...
...
#N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_response(prompt)

IPython.display.Markdown(response)

Here's another example, where there are no steps in the prompt. Let's see how the model worls this time.

In [ ]:

text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

STEPS
#1 - ...
#2 - ...
...
#N - ...

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_response(prompt)

IPython.display.Markdown(response)

#### Tip #4: Give Examples

When you give your model some examples of expected output, you train its output in the direction you want it to take. This is called **One-shot** or **Few-shot** prompting. We will discuss about them in detail in the labs that follow, but let's look at an example right away.

In [ ]:

prompt = f"""
Your task is to answer in a consistent style.

: Teach me about patience.

: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

: Teach me about resilience.
"""

response = get_response(prompt)

IPython.display.Markdown(response)

### 2: Give the model time to “think”

Any LLM would take some time to process your request, search its embeddings to find the most likelyn response, generate the response one token at a time, and return the result back to you. This all takes time, and if you don;t give it the time to "think", you'll make the mistake of overwhelming it by repeated requests.

Here are a couple of tips to expedite its work and help it think the "right way."

#### Tip #1: Specify the steps required to complete a task

Relying on the model to figure out the best way to complete a task will only add to its burden. You can take some responsibility by telling it the steps to follow and speed up content generation. Here's how you can do it:

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_response(prompt_1)

IPython.display.Markdown(response)

##### Ask for output in a specified format

By combining the Tip #2 from above with this tip, you can further simplify the work of the model.

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text:
Text:
```{text}```
"""

response = get_response(prompt_2)

IPython.display.Markdown(response)

#### Tip #2: Instruct to double-check

You can always tell the model to work out its own solution, double chek it, before rushing to a conclusion.

In [ ]:
prompt_1 = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs
250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat
10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_response(prompt_1)

IPython.display.Markdown(response)

You can see for yourself that the solution is not up to the mark - may be its outright wrong, or some aspects are misstated.

We can fix this by instructing the model to work out its own solution first.

In [ ]:
prompt_2 = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs
250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat
10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

response = get_response(prompt_2)

IPython.display.Markdown(response)

## Experiment more to know more!

To use the OpenAI API outside of this classroom

To install the OpenAI Python library:

```
!pip install openai
```
The library needs to be configured with your account's secret key, which is available in your OpenAI account.

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:

```
!export OPENAI_API_KEY='sk-...'
```
Or, set variable `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

**A note about the backslash**

In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters within our texts or prompts.
GPT-3 isn't really affected whether you insert newline characters or not. But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.